In [2]:
# loading data into a dataframe to start validating and cleaning
import pandas as pd
historic_df = pd.read_csv(r"C:\Users\jakes\OneDrive\Desktop\Github\Data\historic-trading-data\historic_data.csv")

In [3]:
# checking which columns have null values
print(historic_df.isnull().sum())

ticker                128
open                    0
high                    0
low                     0
close                   0
volume                  0
vwap                11281
timestamp               0
transactions        11277
otc               1446604
trading_date            0
retrieved_date          0
dtype: int64


In [4]:
# checking null ticker data - appears to relate to one isolated stock based on consistancy of the price (perhaps a summary from polygon without a ticker or corrupted by polygon's api)
# decision is to drop these rows, as it only relates to 128 rows from the million+ currently in the dataframe 
print(historic_df[historic_df['ticker'].isna()])

        ticker   open    high     low  close   volume    vwap      timestamp  \
9626       NaN  4.860  4.9700  4.5000   4.85  10465.0  4.7736  1745956800000   
24819      NaN  4.730  4.9000  4.4700   4.89   7698.0  4.7437  1746043200000   
41279      NaN  4.710  4.8900  4.7100   4.78   1818.0  4.7595  1746129600000   
55925      NaN  4.830  4.9700  4.5709   4.86   9993.0  4.8131  1746216000000   
65688      NaN  4.850  4.9562  4.6700   4.68   2524.0  4.7755  1746475200000   
...        ...    ...     ...     ...    ...      ...     ...            ...   
1910972    NaN  4.835  4.9900  4.6101   4.83  52193.0  4.8040  1761249600000   
1922978    NaN  4.890  4.9475  4.7800   4.87  48210.0  4.8396  1761336000000   
1938092    NaN  4.830  4.9400  4.7401   4.78  34731.0  4.8256  1761595200000   
1957700    NaN  4.780  4.8085  4.6300   4.77  37035.0  4.7206  1761681600000   
1975653    NaN  4.700  4.7600  4.5300   4.65  27560.0  4.6249  1761768000000   

         transactions  otc trading_date

In [5]:
print(len(historic_df.query("vwap.isna() and transactions.isna()")))

11277


In [6]:
# confirms the 11277 instances where transactions and VWAP are null are related to the volume on those days being 0
print(historic_df.query("transactions.isna() and volume > 0"))

Empty DataFrame
Columns: [ticker, open, high, low, close, volume, vwap, timestamp, transactions, otc, trading_date, retrieved_date]
Index: []


In [7]:
# confirms instances with null vwap but without null transactions relate to low marketcap stocks which the polygon API did not obtain the trading prices for
print(historic_df.query("vwap.isna() and transactions.notna()"))

       ticker  open  high  low  close  volume  vwap      timestamp  \
164641  AETHF   0.0   0.0  0.0    0.0   300.0   NaN  1747166400000   
217629  RMANF   0.0   0.0  0.0    0.0  4275.0   NaN  1747684800000   
543009  AETHF   0.0   0.0  0.0    0.0   800.0   NaN  1750276800000   
573100  AETHF   0.0   0.0  0.0    0.0   200.0   NaN  1750708800000   

        transactions   otc trading_date              retrieved_date  
164641           1.0  True   2025-05-13  2025-10-30 14:03:46.110134  
217629           1.0  True   2025-05-19  2025-10-30 14:05:11.254992  
543009           1.0  True   2025-06-18  2025-10-30 14:12:32.926011  
573100           1.0  True   2025-06-23  2025-10-30 14:13:14.949956  


In [8]:
# retaining NULL transactions and VWAP - filtered at VIEW level in SQL
# preserves distinction between true zero and unknown/missing values

# dropping OTC column as lack of value for the planned analysis based only on a boolean value
historic_df = historic_df.drop(['otc'], axis='columns')
# dropping null tickers as there is no value to be gained from these
historic_df = historic_df.dropna(subset=['ticker'])

# saving the cleaned historic data
historic_df.to_csv('historic_data.csv', index=False)